In [1]:
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings coming from Arrow optimizations.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('explore_spark').config('spark.master', 'local').getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
# %timeit ps.range(300000).to_pandas()

## Carregar dados dos ratings/votos dos títulos

In [3]:
title_ratings = spark.read.load('../Data/title.ratings.tsv', format='csv', sep='\t', inferSchema=True, header=True)

In [4]:
title_ratings.show(2)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2007|
|tt0000002|          5.8|     270|
+---------+-------------+--------+
only showing top 2 rows



## Carregar dados dos títulos e concatena os ratings/votos com as informações dos titulos
- somente aqueles que possuem rating
- somente filmes
- somente os que possuem algum gênero

In [5]:
title_basics = spark.read.load('../Data/title.basics.tsv', format='csv', sep='\t', inferSchema=True, header=True)

In [6]:
# title_basics.select(col('titleType')).distinct().show()
title_basics.groupBy('titleType').count().orderBy(col('count').desc()).show()

+------------+-------+
|   titleType|  count|
+------------+-------+
|   tvEpisode|7909714|
|       short| 964958|
|       movie| 664652|
|       video| 283511|
|    tvSeries| 253097|
|     tvMovie| 143596|
|tvMiniSeries|  51367|
|   tvSpecial|  44591|
|   videoGame|  36627|
|     tvShort|  10084|
|     tvPilot|      1|
+------------+-------+



In [7]:
title_basics = title_basics.filter(title_basics['titleType'] == 'movie')
title_basics = title_basics.filter(title_basics['genres'] != '\\N')

In [8]:
# Realiza um join usando 'tconst' como chave e 'inner' como tipo de join
# https://sparkbyexamples.com/pyspark/pyspark-join-explained-with-examples/
title_basics_filtered = title_basics.join(title_ratings, ['tconst'], 'inner').drop('titleType', 'endYear')

In [9]:
title_basics.count()

590391

In [10]:
title_basics_filtered.count()

290648

In [11]:
title_basics_filtered.groupBy('isAdult').count().show()

+-------+------+
|isAdult| count|
+-------+------+
|      0|286094|
|      1|  4554|
+-------+------+



In [12]:
# Criação da nova coluna com a condição de igualdade
title_basics_filtered = title_basics_filtered.withColumn('popularIsOriginal', when(title_basics_filtered['primaryTitle'] == title_basics_filtered['originalTitle'], 1).otherwise(0))

In [13]:
# Filtrar as linhas onde 'runtimeMinutes' é igual a '\N' e contar o número de ocorrências
title_basics_filtered.filter(title_basics_filtered['runtimeMinutes'] == '\\N').count()

25405

In [14]:
title_basics_filtered.show()

+---------+--------------------+--------------------+-------+---------+--------------+--------------------+-------------+--------+-----------------+
|   tconst|        primaryTitle|       originalTitle|isAdult|startYear|runtimeMinutes|              genres|averageRating|numVotes|popularIsOriginal|
+---------+--------------------+--------------------+-------+---------+--------------+--------------------+-------------+--------+-----------------+
|tt0000009|          Miss Jerry|          Miss Jerry|      0|     1894|            45|             Romance|          5.3|     208|                1|
|tt0000147|The Corbett-Fitzs...|The Corbett-Fitzs...|      0|     1897|           100|Documentary,News,...|          5.3|     485|                1|
|tt0000574|The Story of the ...|The Story of the ...|      0|     1906|            70|Action,Adventure,...|          6.0|     855|                1|
|tt0000591|    The Prodigal Son|   L'enfant prodigue|      0|     1907|            90|               Drama

In [15]:
title_basics_filtered.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)
 |-- popularIsOriginal: integer (nullable = false)



## dumificar 'genres'

In [16]:
# Dividir a coluna 'genres' por vírgulas e expandir em colunas
genres_split = title_basics_filtered.withColumn('genres', split('genres', ','))

# Usar a função explode() para criar múltiplas linhas para cada gênero
genres_exploded = genres_split.withColumn('genre', explode('genres'))

# Criar dummies para cada gênero usando pivot()
dummies = genres_exploded.groupBy('tconst').pivot('genre').agg(lit(1)).fillna(0)

# Mostrar o DataFrame resultante com as colunas de dummies para gêneros
dummies.show(5)

+---------+------+-----+---------+---------+---------+------+-----+-----------+-----+------+-------+---------+-------+------+-----+-------+-------+----+----------+-------+------+-----+---------+--------+---+-------+
|   tconst|Action|Adult|Adventure|Animation|Biography|Comedy|Crime|Documentary|Drama|Family|Fantasy|Film-Noir|History|Horror|Music|Musical|Mystery|News|Reality-TV|Romance|Sci-Fi|Sport|Talk-Show|Thriller|War|Western|
+---------+------+-----+---------+---------+---------+------+-----+-----------+-----+------+-------+---------+-------+------+-----+-------+-------+----+----------+-------+------+-----+---------+--------+---+-------+
|tt0000009|     0|    0|        0|        0|        0|     0|    0|          0|    0|     0|      0|        0|      0|     0|    0|      0|      0|   0|         0|      1|     0|    0|        0|       0|  0|      0|
|tt0000147|     0|    0|        0|        0|        0|     0|    0|          1|    0|     0|      0|        0|      0|     0|    0|     

In [17]:
dummies.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- Action: integer (nullable = true)
 |-- Adult: integer (nullable = true)
 |-- Adventure: integer (nullable = true)
 |-- Animation: integer (nullable = true)
 |-- Biography: integer (nullable = true)
 |-- Comedy: integer (nullable = true)
 |-- Crime: integer (nullable = true)
 |-- Documentary: integer (nullable = true)
 |-- Drama: integer (nullable = true)
 |-- Family: integer (nullable = true)
 |-- Fantasy: integer (nullable = true)
 |-- Film-Noir: integer (nullable = true)
 |-- History: integer (nullable = true)
 |-- Horror: integer (nullable = true)
 |-- Music: integer (nullable = true)
 |-- Musical: integer (nullable = true)
 |-- Mystery: integer (nullable = true)
 |-- News: integer (nullable = true)
 |-- Reality-TV: integer (nullable = true)
 |-- Romance: integer (nullable = true)
 |-- Sci-Fi: integer (nullable = true)
 |-- Sport: integer (nullable = true)
 |-- Talk-Show: integer (nullable = true)
 |-- Thriller: integer (nullable = tru

## Carregar dados dos atores principais/equipe dos títulos
- somente dos titulos que possuem rating
- somente filmes
- somente os que possuem algum gênero
- somente atores/atrizes

In [18]:
title_principals = spark.read.load('../Data/title.principals.tsv', format='csv', sep='\t', inferSchema=True, header=True)

In [19]:
title_principals.count()

59388712

In [20]:
# Obter os 'tconst' do DataFrame movies
tconst_movies = title_basics_filtered.select('tconst')

# Filtrar o DataFrame title_principals para manter apenas as linhas em que 'tconst' está presente em movies
title_principals_filtered = title_principals.join(tconst_movies, 'tconst', 'inner').drop('job', 'ordering', 'characters')

In [21]:
title_principals_filtered.count()

2601778

In [22]:
title_principals_filtered.show()

+---------+---------+---------------+
|   tconst|   nconst|       category|
+---------+---------+---------------+
|tt0000941|nm0034453|          actor|
|tt0000941|nm0140054|          actor|
|tt0000941|nm0243918|          actor|
|tt0000941|nm0294022|        actress|
|tt0000941|nm0063413|       director|
|tt0000941|nm0550220|       director|
|tt0000941|nm0848502|         writer|
|tt0002026|nm0115982|          actor|
|tt0002026|nm0418086|        actress|
|tt0002026|nm0027708|          actor|
|tt0002026|nm0526167|        actress|
|tt0002026|nm0959066|          actor|
|tt0002026|nm0259235|       director|
|tt0002026|nm0084804|       producer|
|tt0002026|nm0064944|          actor|
|tt0002026|nm0348052|        actress|
|tt0002026|nm0959065|        actress|
|tt0002423|nm0913298|cinematographer|
|tt0002423|nm0624470|        actress|
|tt0002423|nm0417837|          actor|
+---------+---------+---------------+
only showing top 20 rows



In [23]:
title_principals_filtered.select('category').distinct().show()

+-------------------+
|           category|
+-------------------+
|            actress|
|           producer|
|             writer|
|           composer|
|           director|
|               self|
|              actor|
|             editor|
|    cinematographer|
|      archive_sound|
|production_designer|
|    archive_footage|
+-------------------+



In [24]:
# separar as categorias
actors = ['actress', 'actor', 'self']
producers = ['writer', 'director', 'producer']
crew = ['composer', 'editor', 'cinematographer', 'archive_sound', 'production_designer', 'archive_footage']

title_principals_filtered_actors = title_principals_filtered.filter(col('category').isin(actors))
title_principals_filtered_producers = title_principals_filtered.filter(col('category').isin(producers))
title_principals_filtered_crew = title_principals_filtered.filter(col('category').isin(crew))

In [25]:
title_principals_filtered_actors.count()

1183790

In [26]:
title_principals_filtered_producers.count()

856025

In [27]:
title_principals_filtered_crew.count()

561963

In [28]:
title_principals_filtered_actors.show(2)

+---------+---------+--------+
|   tconst|   nconst|category|
+---------+---------+--------+
|tt0000630|nm0624446| actress|
|tt0000941|nm0034453|   actor|
+---------+---------+--------+
only showing top 2 rows



## Fazer a media de cada grupo pros filmes

In [29]:
from pyspark.sql.functions import avg

# Etapa 1: Obter todos os títulos associados aos nomes presentes no dataset title_principals_filtered_actors
titles_for_names = title_principals_filtered_actors.select('nconst', 'tconst').distinct()

# Etapa 2: Calcular as médias de 'averageRating' e 'numVotes' para esses títulos
avg_ratings_votes = titles_for_names.join(title_basics_filtered, 'tconst', 'inner') \
    .groupBy('nconst') \
    .agg(avg('averageRating').alias('avgRating'), avg('numVotes').alias('avgNumVotes'))

# Mostrar o DataFrame resultante com as médias calculadas
avg_ratings_votes.show()

+---------+------------------+------------------+
|   nconst|         avgRating|       avgNumVotes|
+---------+------------------+------------------+
|nm0000847| 5.836363636363637| 92.13636363636364|
|nm0709856| 5.933333333333333|160.66666666666666|
|nm0689160| 4.333333333333333|36.333333333333336|
|nm0733443|               6.2|29.333333333333332|
|nm0408172| 6.175000000000001|              75.0|
|nm0658029| 5.866666666666667|30.666666666666668|
|nm0392340| 6.128571428571428|233.42857142857142|
|nm0493491|               7.4|            8312.0|
|nm0107574| 6.148148148148148|301.75925925925924|
|nm0849800|              6.25|              27.0|
|nm0151964| 6.045454545454546| 61.09090909090909|
|nm0269647|             6.275|            760.25|
|nm0706978|              5.75|           179.625|
|nm0018091|6.0032258064516135| 87.41935483870968|
|nm0564706| 6.166666666666667|             126.0|
|nm0789419|               6.4|             160.0|
|nm0932629| 5.936111111111112| 653.4722222222222|


In [30]:
# Etapa 1: Obter todos os nomes associados a cada filme no title_basics_filtered
names_for_titles = title_principals_filtered_actors.select('tconst', 'nconst').distinct()

# Etapa 2: Juntar esses nomes com as médias já calculadas
joined_data = avg_ratings_votes.join(names_for_titles, 'nconst', 'inner')

# Etapa 3: Calcular as médias dos filmes com base nos nomes associados
avg_ratings_per_film = joined_data.groupBy('tconst') \
    .agg(avg('avgRating').alias('avgRatingPerFilm'), avg('avgNumVotes').alias('avgNumVotesPerFilm'))

# Mostrar o DataFrame resultante com as médias calculadas para cada filme
avg_ratings_per_film.show()

+---------+------------------+------------------+
|   tconst|  avgRatingPerFilm|avgNumVotesPerFilm|
+---------+------------------+------------------+
|tt0090955| 6.665793402289146| 3845.675810787832|
|tt0036323| 6.234864333057166| 9289.148685618613|
|tt0039305| 6.651033981667784| 7904.853789403086|
|tt0021156| 6.370713181420985|3430.0409271175145|
|tt0058413| 6.401197756865973|2272.7633587143605|
|tt0036277|6.5467464341917925|  4537.81649930474|
|tt0045031| 6.308482142857143| 7868.944196428572|
|tt0032599| 6.514374271561771|7326.7717942383715|
|tt0054152| 6.539334180312441|15050.168534726143|
|tt0054412| 6.349295634920635|17177.531448412698|
|tt0034269|6.3825033068783075| 5789.548613955737|
|tt0035769| 6.440154483953469|4775.8472975620825|
|tt0047072| 6.243173532132557| 1886.574114382108|
|tt0070077| 6.084701213818861|10357.355666433241|
|tt0063210| 6.161137650085763|5064.3422234133795|
|tt0046949| 6.397844537815127|3986.6031123560538|
|tt0027462|6.1392593198168735|3051.2801133638545|


In [31]:
# Selecionar apenas a coluna 'tconst' e coletar os valores
tconst_values = title_basics_filtered.select('tconst').collect()

# Converter os valores coletados em uma lista Python
filmes_list = [row['tconst'] for row in tconst_values]

In [32]:
print('quantidade total de filmes: ', len(filmes_list))

# para cada filme
for filme in filmes_list:

    print('filme: ', filme)

    ##########################################################################################
    #seleciona os atores
    atores = title_principals_filtered_actors.filter(col('tconst') == filme)

    nconst_values = atores.select('nconst').collect()
    atores_list = [row['nconst'] for row in nconst_values]
    
    print(atores_list)

    # separa todos os dados desses atores, para coletar os filmes deles
    atores_completo = title_principals_filtered_actors.filter(col('nconst').isin(atores_list))

    # pega os IDs unicos dos filmes que esses atores participaram
    # Selecionar a coluna 'tconst' e encontrar os filmes únicos
    filmes_unicos = atores_completo.select('tconst').distinct().collect()

    # Converter os filmes coletados em uma lista Python
    lista_filmes_unicos = [row['tconst'] for row in filmes_unicos]

    # isolar esses filmes no dataframe com as notas/votos e fazer a média deles
    dados_filmes_unicos = title_basics_filtered.filter(col('tconst').isin(lista_filmes_unicos))

    
    atores.show()
    break

    ##########################################################################################


quantidade total de filmes:  290648
filme:  tt0000009
['nm0063086', 'nm0183823', 'nm1309758']
+---------+---------+--------+
|   tconst|   nconst|category|
+---------+---------+--------+
|tt0000009|nm0063086| actress|
|tt0000009|nm0183823|   actor|
|tt0000009|nm1309758|   actor|
+---------+---------+--------+



In [33]:
title_basics_filtered.show(2)

+---------+--------------------+--------------------+-------+---------+--------------+--------------------+-------------+--------+-----------------+
|   tconst|        primaryTitle|       originalTitle|isAdult|startYear|runtimeMinutes|              genres|averageRating|numVotes|popularIsOriginal|
+---------+--------------------+--------------------+-------+---------+--------------+--------------------+-------------+--------+-----------------+
|tt0000009|          Miss Jerry|          Miss Jerry|      0|     1894|            45|             Romance|          5.3|     208|                1|
|tt0000147|The Corbett-Fitzs...|The Corbett-Fitzs...|      0|     1897|           100|Documentary,News,...|          5.3|     485|                1|
+---------+--------------------+--------------------+-------+---------+--------------+--------------------+-------------+--------+-----------------+
only showing top 2 rows



## Analisar titulos localizados

In [35]:
title_akas = spark.read.load('../Data/title.akas.tsv', format='csv', sep='\t', inferSchema=True, header=True)

In [36]:
title_akas.show()

+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|  titleId|ordering|               title|region|language|      types|          attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|tt0000001|       1|          Карменсіта|    UA|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       2|          Carmencita|    DE|      \N|         \N|       literal title|              0|
|tt0000001|       3|Carmencita - span...|    HU|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       4|          Καρμενσίτα|    GR|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       5|          Карменсита|    RU|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       6|          Carmencita|    US|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       7|          Carmenc

In [38]:
# Obter os 'tconst' do DataFrame movies
# tconst_movies contém os IDs de filmes já selecionados

# filtrar somente as entradas de titulos que estamos considerando
title_akas_filtered = title_akas.join(tconst_movies, title_akas['titleId'] == tconst_movies['tconst'], 'inner')

In [39]:
title_akas_filtered.show()

+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+---------+
|  titleId|ordering|               title|region|language|      types|          attributes|isOriginalTitle|   tconst|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+---------+
|tt0000941|       1|      Locura de amor|    \N|      \N|   original|                  \N|              1|tt0000941|
|tt0000941|       2|          Love Crazy|   XWW|      en|         \N|informal literal ...|              0|tt0000941|
|tt0000941|       3|      Locura de amor|    ES|      \N|imdbDisplay|                  \N|              0|tt0000941|
|tt0002026|       1|Anny - en gatepig...|    NO|      \N|imdbDisplay|                  \N|              0|tt0002026|
|tt0002026|       2|Anny - Story of a...|   XWW|      en|         \N|                  \N|              0|tt0002026|
|tt0002026|       3|Anny - en gatepig...|    \N|      \N|   orig

In [45]:
title_akas_filtered_translations = title_akas_filtered.groupBy('tconst').count().withColumnRenamed('count', 'numberOfTranslations')
title_akas_filtered_translations.show()

+---------+--------------------+
|   tconst|numberOfTranslations|
+---------+--------------------+
|tt0000147|                   3|
|tt0000574|                   9|
|tt0000630|                   5|
|tt0000675|                   2|
|tt0000886|                   4|
|tt0000941|                   3|
|tt0001028|                   2|
|tt0001049|                   3|
|tt0001112|                   4|
|tt0001115|                   2|
|tt0001184|                   2|
|tt0001277|                   2|
|tt0001338|                   3|
|tt0001370|                   2|
|tt0001440|                   3|
|tt0001498|                   4|
|tt0001530|                   3|
|tt0001790|                   6|
|tt0001812|                   2|
|tt0001911|                   5|
+---------+--------------------+
only showing top 20 rows



## Agregar dataset de features

In [44]:
title_basics_filtered.show()

+---------+--------------------+--------------------+-------+---------+--------------+--------------------+-------------+--------+-----------------+
|   tconst|        primaryTitle|       originalTitle|isAdult|startYear|runtimeMinutes|              genres|averageRating|numVotes|popularIsOriginal|
+---------+--------------------+--------------------+-------+---------+--------------+--------------------+-------------+--------+-----------------+
|tt0000009|          Miss Jerry|          Miss Jerry|      0|     1894|            45|             Romance|          5.3|     208|                1|
|tt0000147|The Corbett-Fitzs...|The Corbett-Fitzs...|      0|     1897|           100|Documentary,News,...|          5.3|     485|                1|
|tt0000574|The Story of the ...|The Story of the ...|      0|     1906|            70|Action,Adventure,...|          6.0|     855|                1|
|tt0000591|    The Prodigal Son|   L'enfant prodigue|      0|     1907|            90|               Drama

In [48]:
titles_features_complete = title_basics_filtered.join(title_akas_filtered_translations, 'tconst', 'inner')
titles_features_complete = titles_features_complete.join(dummies, 'tconst', 'inner')

In [49]:
titles_features_complete.show()

+---------+--------------------+--------------------+-------+---------+--------------+--------------------+-------------+--------+-----------------+--------------------+------+-----+---------+---------+---------+------+-----+-----------+-----+------+-------+---------+-------+------+-----+-------+-------+----+----------+-------+------+-----+---------+--------+---+-------+
|   tconst|        primaryTitle|       originalTitle|isAdult|startYear|runtimeMinutes|              genres|averageRating|numVotes|popularIsOriginal|numberOfTranslations|Action|Adult|Adventure|Animation|Biography|Comedy|Crime|Documentary|Drama|Family|Fantasy|Film-Noir|History|Horror|Music|Musical|Mystery|News|Reality-TV|Romance|Sci-Fi|Sport|Talk-Show|Thriller|War|Western|
+---------+--------------------+--------------------+-------+---------+--------------+--------------------+-------------+--------+-----------------+--------------------+------+-----+---------+---------+---------+------+-----+-----------+-----+------+--

## Salvar os dados selecionados

In [1]:
# Salvar como CSV
titles_features_complete.write.csv('../Data/movies.features.complete.csv', header=True)

NameError: name 'titles_features_complete' is not defined